In [1]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests

In [2]:
def get_links_by_state():
    q = "https://www.craigslist.org/about/sites#US"
    html = requests.get(q).text
    soup = bs(html, 'html.parser')
    p_list = soup.find_all('div')
    hrefs = [p for p in p_list if p.get('class') == [u'colmask']]
    hrefs = str(hrefs[0]).split('<h4>')
    hrefs2 = []
    for h in hrefs:
        hrefs2.append(h.split('</h4>'))
    states = {}
    for html_split in hrefs2:
        if len(html_split) == 2:
            states[html_split[0].lower()] = html_split[1]
    for state in states.keys():
        soup = bs(states[state], 'html.parser')
        links = soup.find_all('a')
        states[state] = [get_city_from_url(a.get("href")) for a in links]
    return states

def get_car_links(url):
    city = get_city_from_url(url)
    html = requests.get(url).text
    soup = bs(html, 'html.parser')
    link_list = soup.find_all('a') 
    car_list = [a for a in soup.find_all('a') 
                 if str(a.get('class')) == "[u'hdrlnk']"]
                 #if any(x in str(a) for x in search_strings)]
    text_list, href_list = [], []
    for i, l in enumerate(car_list):
        text_list.append(unicode(l.contents[0]))
        href = l.get('href')
        if href.startswith("//"):
            href = "http:{0}".format(href)
        elif href.startswith("http"):
            pass
        else:
            href = "http://{0}.craigslist.org{1}".format(city, href)
        href_list.append(href)
    link_rels = soup.find_all('link')
    try:
        next_page = [a.get('href') for a in link_rels if a.get('rel')[0] == u'next'][0]
    except IndexError:
        return text_list, href_list
    tl, hl = get_car_links(next_page)
    text_list+= tl
    href_list += hl
    return text_list, href_list

def get_city_from_url(url):
    return url.split('/')[2].split('.')[0]

def get_attrs(lnk):
    html = requests.get(lnk).text
    soup = bs(html, 'html.parser')
    p_list = soup.find_all('p')
    spans = soup.find_all('span')
    try:
        price = [s for s in spans if s.get('class') == [u'price']][0].contents[0]
    except:
        price = np.NaN
    attrs = [p for p in p_list if p.get('class') == [u"attrgroup"]]
    attr_list = attrs[1].find_all('span')
    d = {}
    d['Price'] = price
    for attr in attr_list:
        try:
            d[attr.contents[0]] = attr.contents[1].contents[0]
        except IndexError:
            pass
    return d

In [3]:
states = get_links_by_state()
#state_list = ['texas', 'arkansas', 'alabama', 'mississippi', 'new mexico', 'colorado', 'oklahoma', 'louisiana', 'kansas']
state_list = ['georgia']
city_list = []
for state in state_list:
    city_list += states[state]
print city_list
search_strings = ['2009', '2010', '2011', '2012', '2013', '2014', '2015']
stop_strings = ['gti', 'chevrolet', 'chevy', 'ford', '3.0', 'touareg', 'nissan', 'gmc', 'chrysler',
                'dodge', 'saturn', 'mercedes', 'honda', 'buick', 'toyota', 'jeep', 'lincoln',
                'scion', 'yamaha', ]
#city_list = ['dallas', 'houston', 'austin']
#city = 'austin'
#url = "https://{}.craigslist.org/search/cta?query=tdi".format(city)

[u'albanyga', u'athensga', u'atlanta', u'augusta', u'brunswick', u'columbusga', u'macon', u'nwga', u'savannah', u'statesboro', u'valdosta']


In [4]:
ds = []
city_list = ['dallas']
for city in city_list:
    url = "https://{}.craigslist.org/search/cta?query=tdi".format(city)
    tlst, hlst = get_car_links(url)
    if tlst and hlst:
        df = pd.DataFrame(zip(tlst, hlst), columns=['Title', 'Link'])
        df = df[df['Title'].str.lower().str.contains('|'.join(search_strings))]
        df = df[~df['Title'].str.lower().str.contains('|'.join(stop_strings))]
        for lnk, title in zip(df['Link'].values, df['Title'].values):
            d = get_attrs(lnk)
            d['Link'] = lnk
            d['Title'] = title
            ds.append(d)
car_data = pd.DataFrame(ds)

In [5]:
len(car_data.index)

196

In [6]:
car_data.to_csv('raw_car_data.csv', index=False, encoding='utf-8')

In [7]:
car_data = pd.read_csv('raw_car_data.csv')

In [6]:
cleaned_car_data = car_data.drop_duplicates()
print(len(cleaned_car_data.index))
cleaned_car_data = cleaned_car_data[cleaned_car_data[u'title status: ']=='clean']
len(cleaned_car_data.index)

54


52

In [7]:
cleaned_car_data.to_csv('cleaned_car_data.csv', index=False, encoding='utf-8')